# TCP comms

In [1]:
import socket

In [3]:
TCP_IP = '10.101.241.28'
TCP_PORT = 5005
BUFFER_SIZE = 1024
MESSAGE = "Hello, World!"

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((TCP_IP, TCP_PORT))
s.send(MESSAGE)
data = s.recv(BUFFER_SIZE)
s.close()

# HEX converter

In [10]:
def pretty_hex(num):
    return hex(num)[2:-1]

# DB meter

In [4]:
import pyaudio
import numpy as np
import audioop
import pandas as pd

CHUNK = 4096 # number of data points to read at a time
RATE = 44100 # time resolution of the recording device (Hz)


AttributeError: 'module' object has no attribute 'PyAudio'

In [11]:

p=pyaudio.PyAudio() # start the PyAudio class
stream=p.open(format=pyaudio.paInt16,channels=2,rate=RATE,input=True,
              frames_per_buffer=CHUNK, input_device_index = 1) #uses default input device

# generate RMS for the 10 samples

# for proper val compared to background noise need to get samples from mic2 and divide by it


DB = []

for i in range(10):
    
    data = []
    for i in range(10): #to it a few times just to see
        data = np.fromstring(stream.read(CHUNK),dtype=np.int16).astype(np.float)
    
        
        data[data <= 0.0000000001] = 0

    # the rms gives us the power from the amplitude
        
        db = 20 * np.log10(audioop.rms(data, 2))
        DB.append(db)
        print pretty_hex(db)
    
        # convert to amplitude
#         data = np.divide(data, float(32767))
        
        
        # issue we are taking the log 10 of numbers this small makes numpy throw error
        
#         mic1 = data.mean()
        
#         db.append( 20 * np.log10(mic1))
            

pd.DataFrame(data).plot(title = "Raw data")
pd.DataFrame(DB).plot(title = "Decibels 0 = quiet > 80 = factory level noise")
stream.stop_stream()
stream.close()
p.terminate()

4f
4f
4f
4e
4f
4f
4e
4f
4e
4f
4e
4f
4e
4f
4f
4e
4e
4f
4e
4e
4f
4e
4f
4e
4f
4e
4e
4e
4e
4f
4e
4e
4f
4e
4f
4f
4e
4e
4f
4e
4e
4f
4e
4f
4e
4f
4f
4e
4e
4f
4f
4e
4e
4e
4f
4e
4f
4e
4e
4e
4f
4f
4e
4f
4f
4e
4f
4f
4e
4e
4e
4f
4e
4f
4e
4e
4e
4f
4f
4e
4f
4f
4e
4f
4f
4f
4e
4f
4e
4e
4f
4e
4f
4e
4e
4f
4e
4f
4f
4f


# putting everything together

In [33]:
import pyaudio
import numpy as np
import audioop
import pandas as pd

CHUNK = 4096 # number of data points to read at a time
RATE = 44100 # time resolution of the recording device (Hz)

import socket

def pretty_hex(num):
    hex_num = hex(num)[2:]
    
    if hex_num[-1] == 'L': hex_num = hex_num[:-1]
    return hex_num
    
    
def encode(ID, time_sent, DB, Distances):
    
    data = [ID, time_sent, DB] + Distances
    return '_'.join(data)

import time
import errno

def transmit(data):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect(('127.0.0.1', 81))
        s.send(data)
    
        while(s.recv(2048) != "ack"):

            print "waiting for acknowledgment"
    
    except socket.error as error:
#         print error.errno
        print "tcp error"
        if error.errno == errno.WSAECONNRESET:
            
#         reconnect()
#         retry_action()
            print "not connected to server"
    
    finally:
#     else:
#         raise
        s.close()
    
def record_db():
    p=pyaudio.PyAudio() # start the PyAudio class
    stream=p.open(format=pyaudio.paInt16,channels=2,rate=RATE,input=True,
              frames_per_buffer=CHUNK, input_device_index = 1) #uses default input device

    data = np.fromstring(stream.read(CHUNK),dtype=np.int16).astype(np.float)

    # omit outliers
    data[data <= 0.0000000001] = 0

    
    # the rms gives us the power from the amplitude?
    # for proper val compared to background noise need to get samples from mic2 and divide by it

    db = 20 * np.log10(audioop.rms(data, 2))
#     DB.append(db)
   
    stream.stop_stream()
    stream.close()
    p.terminate()
    return db

def get_Box_ID():
    f = open("Box_ID.txt", 'r')
    return f.read()

In [7]:
## just in case Im stuck with HTTP
import urllib2
def http_transmit(data):
    urllib2.urlopen("http://seat-skomobo.massey.ac.nz/" + data)

In [42]:
ID = pretty_hex(int(get_Box_ID()))
db = pretty_hex(record_db())
distances = map(pretty_hex, [41, 23, 12, 20, 1, 49, 100])
packet = encode(ID, "2016-6-23", db, distances)
transmit(packet)
print packet

waiting for acknowledgment
1_2016-6-23_4e_29_17_c_14_1_31_64
